### Given a currency basket and start time, we estimate the weights and analyse the performance.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import coint_trading as ct
import utils as ut
import plotting as myplt
import data_io as myio
import mean_reversion as mrev
import pandas as pd

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual
from datetime import datetime

from ta.momentum import RSIIndicator

import warnings
warnings.filterwarnings("ignore")

### User Specification

In [2]:
name_list = ['EURUSD Curncy', 'GBPUSD Curncy', 'AUDUSD Curncy', 'NZDUSD Curncy', 'CNHUSD Curncy']



In [12]:
df_raw = pd.read_excel('../data/bloomberg fx hourly data sheet.xlsx')

df_fx_spot = pd.DataFrame()

def one_over(x):
    return 1.0/x

ccy_list = ['CAD', 'JPY', 'CHF', 'SGD', 'CNH']
for ccy in ccy_list:      
    df_temp = df_raw[[ccy, 'USD' + ccy + ' Curncy']]    
    df_temp = df_temp.dropna() 
    df_temp = df_temp.rename(columns={ccy: "Time"})
    df_temp = df_temp.set_index('Time')   
    # convert the column from USDXXX to XXXUSD 
    df_temp = df_temp.rename(columns={'USD' + ccy + ' Curncy': ccy + 'USD Curncy'})
    df_fx_spot = pd.concat([df_fx_spot, df_temp], axis=1)
    
# remove the row when there is no data for any ccy pair, e.g. SG Monday morning at 2:00am in order to avoid 1/x when x = 0.
df_fx_spot = df_fx_spot.dropna()

# convert to XXXUSSD by 1/USDXXX
df_fx_spot = df_fx_spot.apply(one_over, axis=1)

ccy_list = ['EUR', 'GBP', 'AUD', 'NZD']
for ccy in ccy_list:  
    df_temp = df_raw[[ccy, ccy + 'USD Curncy']]
    df_temp = df_temp.dropna()
    df_temp = df_temp.rename(columns={ccy: "Time"})
    df_temp = df_temp.set_index('Time')    
    df_fx_spot = pd.concat([df_fx_spot, df_temp], axis=1)

# remove the row when there is no data for any ccy pair again to make sure all time stamp has data for all ccy pairs.
df_fx_spot = df_fx_spot.dropna()

FROM = df_fx_spot.index[0]
NOW = df_fx_spot.index[-1]
print(FROM)
print(NOW)
print("number of data points:" + ' ' + str(len(df_fx_spot.index)))

2023-10-02 03:00:00
2024-02-14 22:00:00
number of data points: 2336


### Read in data and perform estimation using the latest data

In [14]:
#df_fx_spot = df_fx_spot.loc[FROM:TODAY]
df_fx_name_list_XXXUSD = df_fx_spot[name_list]

res_estimation = ct.johansen_test_estimation(df_fx_name_list_XXXUSD, name_list, 0, 1)
res_diag = ct.johansen_test_diag(res_estimation, df_fx_name_list_XXXUSD, name_list, True, 0, 1)

weights_XXXUSD = res_estimation['weights']
basket = res_diag['Johansen Basket']
#------------------------------------------------------------------------------------
LOWER_THRESHOLD = 2.2
HIGHER_THRESHOLD = 2.5

mean_rev_ts = mrev.MeanRevTimeSeries(basket)
long_run_mean = mean_rev_ts.get_mean_rev_level()
historical_std = mean_rev_ts.get_stdev()

df_MeanRevStats = mrev.compute_sharpe_and_buysell_signal_multi_period(basket,
                                                                      long_run_mean,
                                                                      historical_std,
                                                                      mean_rev_ts.get_mean_rev_rate_in_days(),                                                                      
                                                                      mean_rev_ts.get_half_life_in_days(),
                                                                      LOWER_THRESHOLD,
                                                                      HIGHER_THRESHOLD)

basket_x_days_hist_rtns = ut.compute_x_day_historical_returns(basket)
#------------------------------------------------------------------------------------
df_buy_sell_signal = df_MeanRevStats[['Buy Signal', 'Sell Signal']]
df_Sharpe = df_MeanRevStats[['Sharpe Ratio']]

df_hist_vol = pd.DataFrame(ut.compute_daily_hist_normal_vol(basket, 15))
df_rsi = pd.DataFrame(RSIIndicator(basket, 14).rsi())

shock_in_market_conv_pips = 10
dBasket_SD = ut.basket_SD_change_due_to_fx_spot_change(df_fx_name_list_XXXUSD, 
                                                       weights_XXXUSD, 
                                                       shock_in_market_conv_pips,
                                                       long_run_mean,
                                                       historical_std
                                                       )
#print(dBasket_SD)

ts = (basket - long_run_mean)/historical_std
df_diff = pd.DataFrame(ts.diff())
df_diff = df_diff.rename(columns={"Basket": "One day diff in SD"})
print("--------------------------------------------------")
print(df_diff.quantile([0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]))

 -------------------------------------------------------
                weights  PX_LAST  market convention notional
EURUSD Curncy  1.000000   1.0716                    1.000000
GBPUSD Curncy -0.164138   1.2559                   -0.164138
AUDUSD Curncy -1.248345   0.6480                   -1.248345
NZDUSD Curncy  0.825969   0.6079                    0.825969
CNHUSD Curncy -4.045390   7.2253                    0.559892
USD_amount = 0.0013
 -------------------------------------------------------
Pass trace test (5%) = False
Pass eigen test (5%) = False
Pass trace test (10%) = False
Pass eigen test (10%) = False
          trace      10%       5%       1%
r=0   61.468209  65.8202  69.8189  77.8202
r<=1  37.594725  44.4929  47.8545  54.6815
r<=2  20.909247  27.0669  29.7961  35.4628
r<=3   9.513727  13.4294  15.4943  19.9349
r<=4   2.649953   2.7055   3.8415   6.6349
          eigen      10%       5%       1%
r=0   23.873484  31.2379  33.8777  39.3693
r<=1  16.685478  25.1236  27.5858  32.

In [21]:
DATE = "2023-11-02 03:00:00"

### Time Seires of Basket with z-score using historical std (for trading signal)

In [22]:
@interact_manual
def show_articles_more_than(start_date = widgets.DatePicker(value=pd.to_datetime(FROM)),
                            end_date   = widgets.DatePicker(value=pd.to_datetime(NOW)),
                            vert_line_date = widgets.DatePicker(value=pd.to_datetime(DATE)),
                            z_lower = (-8, 2, 0.5), z_upper = (-2, 8, 0.5), quantile = (1,10)):        
    xlim_range = (start_date, end_date)   
    ylim_range = (long_run_mean + z_lower * historical_std, long_run_mean + z_upper * historical_std)
    myplt.plot_trading_signal_graphs(basket, long_run_mean, historical_std, 
                                     df_buy_sell_signal, xlim_range, ylim_range, vert_line_date) 
    myplt.normal_plot(df_hist_vol, xlim_range, vert_line_date, quantile, 'b-o') 
    myplt.normal_plot(df_Sharpe, xlim_range, vert_line_date, quantile, 'g-o')  
    myplt.normal_plot(df_rsi, xlim_range, vert_line_date, quantile, 'r-o')
    #myplt.normal_plot(df_diff, xlim_range, vert_line_date, quantile, 'k')     

interactive(children=(DatePicker(value=Timestamp('2023-10-02 03:00:00'), description='start_date', step=1), Da…

### PnL Performance

In [ ]:
start_date = '2024-01-03'
end_date = TODAY

traded_fx_rates = df_fx_name_list_XXXUSD.loc[start_date]

num_rows = df_MeanRevStats['Return Expectation over X days'].loc[start_date:end_date].count()

### if using constant notional 
constant_notional = -2000000

# get the size of the list correct
constant_notionals = np.ones(num_rows) * constant_notional

output_xls = '../PnL Analysis from ' + start_date + ' to ' + end_date + '.xlsx'

df_pnl = ut.pnl_analysis_info_to_Excel(df_fx_name_list_XXXUSD, 
                                       constant_notionals, 
                                       weights_XXXUSD,
                                       traded_fx_rates, 
                                       start_date, 
                                       end_date)



df_pnl.to_excel(output_xls)

df_pnl['Basket Cumulative PnL'].plot(grid=True, figsize=(15,5))